# Acquire

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from scipy import stats
import sklearn.preprocessing


from env import user, password, host
import wrangle

In [2]:
def get_db_url(database):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

In [3]:
url = get_db_url('zillow')

In [4]:
acquire_df = pd.read_sql('''
SELECT  propertylandusetypeid
        roomcnt,
        bedroomcnt,
        bathroomcnt,
        fullbathcnt
        garagecarcnt,
        garagetotalsqft,
        yearbuilt,
        taxamount,
        fips,
        calculatedfinishedsquarefeet,
        taxvaluedollarcnt,
        lotsizesquarefeet,
        landtaxvaluedollarcnt,
        poolcnt
FROM properties_2017 
JOIN propertylandusetype USING (propertylandusetypeid)
JOIN predictions_2017 ON properties_2017.id = predictions_2017.id
WHERE propertylandusetype.propertylandusedesc = "Single Family Residential" 
AND predictions_2017.transactiondate LIKE "2017%%"
''', url)

In [5]:
acquire_df.head()

,roomcnt,bedroomcnt,bathroomcnt,garagecarcnt,garagetotalsqft,yearbuilt,taxamount,fips,calculatedfinishedsquarefeet,taxvaluedollarcnt,lotsizesquarefeet,landtaxvaluedollarcnt,poolcnt
0,261.0,3.0,3.0,3.0,NaN,1936.0,7262.79,6037.0,2490.0,594518.0,6729.0,399435.0,NaN
1,261.0,2.0,2.0,2.0,NaN,1938.0,11864.30,6037.0,1600.0,980000.0,10080.0,784000.0,NaN
2,261.0,2.0,2.0,2.0,NaN,1954.0,3888.13,6037.0,1276.0,322119.0,5750.0,142807.0,1.0
3,261.0,3.0,2.0,2.0,NaN,1937.0,5268.54,6037.0,1976.0,435017.0,6235.0,202513.0,1.0
4,261.0,4.0,3.0,3.0,NaN,1946.0,7925.72,6037.0,1998.0,658823.0,5066.0,403546.0,NaN


# Clean/Prep

In [6]:
acquire_df.shape

(56079, 13)

In [7]:
acquire_df.describe().T

,count,mean,std,min,25%,50%,75%,max
roomcnt,56079.0,261.000000,0.000000,261.0,261.00,261.00,261.00,261.00
bedroomcnt,56079.0,3.289110,0.958756,0.0,3.00,3.00,4.00,25.00
bathroomcnt,56079.0,2.239457,1.007012,0.0,2.00,2.00,3.00,20.00
garagecarcnt,55693.0,2.199289,0.973534,1.0,2.00,2.00,3.00,20.00
garagetotalsqft,18247.0,488.088343,162.185770,0.0,420.00,460.00,532.00,3774.00
yearbuilt,55830.0,1961.186262,21.995171,1862.0,1949.00,1958.00,1976.00,2016.00
taxamount,55949.0,5589.363490,8541.810318,8.3,2523.76,4130.48,6410.56,1195295.62
fips,56079.0,6048.841581,21.237610,6037.0,6037.00,6037.00,6059.00,6111.00
calculatedfinishedsquarefeet,55848.0,1868.285704,966.973076,20.0,1260.00,1630.00,2217.00,26345.00
taxvaluedollarcnt,56071.0,457643.446951,663558.094993,9.0,187026.00,329525.00,533484.50,67506489.00


In [8]:
acquire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56079 entries, 0 to 56078
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   roomcnt                       56079 non-null  float64
 1   bedroomcnt                    56079 non-null  float64
 2   bathroomcnt                   56079 non-null  float64
 3   garagecarcnt                  55693 non-null  float64
 4   garagetotalsqft               18247 non-null  float64
 5   yearbuilt                     55830 non-null  float64
 6   taxamount                     55949 non-null  float64
 7   fips                          56079 non-null  float64
 8   calculatedfinishedsquarefeet  55848 non-null  float64
 9   taxvaluedollarcnt             56071 non-null  float64
 10  lotsizesquarefeet             55729 non-null  float64
 11  landtaxvaluedollarcnt         56071 non-null  float64
 12  poolcnt                       10951 non-null  float64
dtypes

In [9]:
# How many nulls are present in our dataset?
acquire_df.isnull().sum()

roomcnt                             0
bedroomcnt                          0
bathroomcnt                         0
garagecarcnt                      386
garagetotalsqft                 37832
yearbuilt                         249
taxamount                         130
fips                                0
calculatedfinishedsquarefeet      231
taxvaluedollarcnt                   8
lotsizesquarefeet                 350
landtaxvaluedollarcnt               8
poolcnt                         45128
dtype: int64

In [10]:
acquire_df.garagecarcnt.value_counts().sum()

55693

In [11]:
acquire_df['garagecarcnt'].value_counts()

2.0     28258
3.0     11896
1.0     11417
4.0      2648
5.0       938
6.0       330
7.0       126
8.0        40
9.0        25
10.0        9
11.0        3
13.0        1
20.0        1
19.0        1
Name: garagecarcnt, dtype: int64

In [12]:
acquire_df.garagecarcnt

0        3.0
1        2.0
2        2.0
3        2.0
4        3.0
        ... 
56074    1.0
56075    2.0
56076    1.0
56077    2.0
56078    2.0
Name: garagecarcnt, Length: 56079, dtype: float64

In [13]:
# What's the percentage of nulls?
acquire_df.isna().mean()

roomcnt                         0.000000
bedroomcnt                      0.000000
bathroomcnt                     0.000000
garagecarcnt                    0.006883
garagetotalsqft                 0.674620
yearbuilt                       0.004440
taxamount                       0.002318
fips                            0.000000
calculatedfinishedsquarefeet    0.004119
taxvaluedollarcnt               0.000143
lotsizesquarefeet               0.006241
landtaxvaluedollarcnt           0.000143
poolcnt                         0.804722
dtype: float64

**Takeaways**
- `poolcnt` has far too many nulls to be used moving forward (drop)
- Because `garagetotalsqft` is composed of 67% null values, I'll drop this variable as well (drop)
- `roomcnt`, `garagecarcnt`, `lotsizesquarefeet`,  all have limited nulls, so I'll move forward

In [14]:
acquire_df.drop(columns=['garagetotalsqft','poolcnt'])

,roomcnt,bedroomcnt,bathroomcnt,garagecarcnt,yearbuilt,taxamount,fips,calculatedfinishedsquarefeet,taxvaluedollarcnt,lotsizesquarefeet,landtaxvaluedollarcnt
0,261.0,3.0,3.0,3.0,1936.0,7262.79,6037.0,2490.0,594518.0,6729.0,399435.0
1,261.0,2.0,2.0,2.0,1938.0,11864.30,6037.0,1600.0,980000.0,10080.0,784000.0
2,261.0,2.0,2.0,2.0,1954.0,3888.13,6037.0,1276.0,322119.0,5750.0,142807.0
3,261.0,3.0,2.0,2.0,1937.0,5268.54,6037.0,1976.0,435017.0,6235.0,202513.0
4,261.0,4.0,3.0,3.0,1946.0,7925.72,6037.0,1998.0,658823.0,5066.0,403546.0
...,...,...,...,...,...,...,...,...,...,...,...
56074,261.0,3.0,1.0,1.0,1951.0,1392.63,6037.0,1131.0,105192.0,7194.0,53171.0
56075,261.0,3.0,2.0,2.0,1942.0,2191.93,6037.0,1354.0,169439.0,9064.0,77337.0
56076,261.0,2.0,1.0,1.0,1949.0,1962.75,6037.0,820.0,148573.0,8007.0,79376.0
56077,261.0,4.0,2.0,2.0,1941.0,2411.01,6037.0,1232.0,190984.0,6270.0,124140.0


In [15]:
# If we drop all rows containing any null, how much data do we lose?
round(acquire_df.dropna().shape[0] / acquire_df.shape[0], 4)

0.0663

In [16]:
# Count of all current features
for column in acquire_df.columns:
    print(column)
    print(acquire_df[column].value_counts())
    print("--------------------")

roomcnt
261.0    56079
Name: roomcnt, dtype: int64
--------------------
bedroomcnt
3.0     25018
4.0     16582
2.0      8743
5.0      3974
6.0       661
1.0       587
0.0       359
7.0       111
8.0        33
9.0         8
25.0        1
10.0        1
11.0        1
Name: bedroomcnt, dtype: int64
--------------------
bathroomcnt
2.0     24535
3.0     11152
1.0     10630
2.5      3723
4.0      2147
1.5       793
5.0       780
3.5       736
4.5       506
0.0       366
6.0       303
5.5       165
7.0       116
8.0        37
6.5        30
9.0        26
10.0       11
7.5        11
8.5         5
11.0        3
13.0        1
9.5         1
20.0        1
19.5        1
Name: bathroomcnt, dtype: int64
--------------------
garagecarcnt
2.0     28258
3.0     11896
1.0     11417
4.0      2648
5.0       938
6.0       330
7.0       126
8.0        40
9.0        25
10.0        9
11.0        3
13.0        1
20.0        1
19.0        1
Name: garagecarcnt, dtype: int64
--------------------
garagetotalsqft
400

**Takeaways So Far**
- Bedrooms can be an integer
- Bathrooms can stay a float to keep the .5 bathrooms
- Year built, FIPS, and taxvaluedollarcnt can be converted to integers w/o data loss

In [17]:
# Dropping nulls due to relative insignificance to overall dataset
acquire_df = acquire_df.dropna()

In [18]:
acquire_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3718 entries, 567 to 55260
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   roomcnt                       3718 non-null   float64
 1   bedroomcnt                    3718 non-null   float64
 2   bathroomcnt                   3718 non-null   float64
 3   garagecarcnt                  3718 non-null   float64
 4   garagetotalsqft               3718 non-null   float64
 5   yearbuilt                     3718 non-null   float64
 6   taxamount                     3718 non-null   float64
 7   fips                          3718 non-null   float64
 8   calculatedfinishedsquarefeet  3718 non-null   float64
 9   taxvaluedollarcnt             3718 non-null   float64
 10  lotsizesquarefeet             3718 non-null   float64
 11  landtaxvaluedollarcnt         3718 non-null   float64
 12  poolcnt                       3718 non-null   float64
dtype

In [19]:
# 100% of calculatedfinishedsquarefeet can be converted to int w/o data loss
(acquire_df.calculatedfinishedsquarefeet == acquire_df.calculatedfinishedsquarefeet.astype(int)).mean()

1.0

In [20]:
# 100% of calculatedfinishedsquarefeet can lose the decimal and be OK
(acquire_df.taxvaluedollarcnt == acquire_df.taxvaluedollarcnt.astype(int)).mean()

1.0

In [21]:
(acquire_df.bathroomcnt == acquire_df.bathroomcnt.astype(int)).mean()

0.6234534696073157

In [22]:
# fips, yearbuilt, and bedrooms can be integers
acquire_df["fips"] = acquire_df["fips"].astype(int)
acquire_df["yearbuilt"] = acquire_df["yearbuilt"].astype(int)
acquire_df["bedroomcnt"] = acquire_df["bedroomcnt"].astype(int)
acquire_df["taxvaluedollarcnt"] = acquire_df["taxvaluedollarcnt"].astype(int)
acquire_df["calculatedfinishedsquarefeet"] = acquire_df["calculatedfinishedsquarefeet"].astype(int)

In [23]:
acquire_df.dtypes

roomcnt                         float64
bedroomcnt                        int64
bathroomcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
yearbuilt                         int64
taxamount                       float64
fips                              int64
calculatedfinishedsquarefeet      int64
taxvaluedollarcnt                 int64
lotsizesquarefeet               float64
landtaxvaluedollarcnt           float64
poolcnt                         float64
dtype: object

`taxamount` is potentially misleading if carried through to analysis. Due to factors such as local/regional tax statutes and unknown time of collection, this data is not a good variable to predict home_value

In [24]:
acquire_df.drop(columns = 'taxamount')

,roomcnt,bedroomcnt,bathroomcnt,garagecarcnt,garagetotalsqft,yearbuilt,fips,calculatedfinishedsquarefeet,taxvaluedollarcnt,lotsizesquarefeet,landtaxvaluedollarcnt,poolcnt
567,261.0,5,4.0,4.0,781.0,1977,6111,4070,1464000,140698.0,1082000.0,1.0
573,261.0,4,3.5,3.0,705.0,1988,6111,2475,826982,4974.0,330792.0,1.0
574,261.0,4,3.0,3.0,529.0,1988,6111,2698,825978,5538.0,412989.0,1.0
578,261.0,4,2.0,2.0,437.0,1970,6111,1833,497696,7577.0,199078.0,1.0
579,261.0,3,2.0,2.0,462.0,1970,6111,1594,698729,8163.0,420821.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
55217,261.0,5,4.5,4.0,625.0,2001,6059,5070,1248282,11579.0,341734.0,1.0
55221,261.0,5,4.5,4.0,665.0,2000,6059,3793,901884,9750.0,348751.0,1.0
55234,261.0,5,5.5,5.0,761.0,2001,6059,7812,3893609,37113.0,2209964.0,1.0
55238,261.0,5,4.5,4.0,646.0,2000,6059,3687,923000,7032.0,491975.0,1.0


In [25]:
acquire_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3718 entries, 567 to 55260
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   roomcnt                       3718 non-null   float64
 1   bedroomcnt                    3718 non-null   int64  
 2   bathroomcnt                   3718 non-null   float64
 3   garagecarcnt                  3718 non-null   float64
 4   garagetotalsqft               3718 non-null   float64
 5   yearbuilt                     3718 non-null   int64  
 6   taxamount                     3718 non-null   float64
 7   fips                          3718 non-null   int64  
 8   calculatedfinishedsquarefeet  3718 non-null   int64  
 9   taxvaluedollarcnt             3718 non-null   int64  
 10  lotsizesquarefeet             3718 non-null   float64
 11  landtaxvaluedollarcnt         3718 non-null   float64
 12  poolcnt                       3718 non-null   float64
dtype

In [26]:
# How many different categories are in fips?
acquire_df['fips'].nunique()

2

In [27]:
acquire_df.fips.value_counts()

6059    2928
6111     790
Name: fips, dtype: int64

fips is akin to county code:
- 6037 = LA county
- 6059 = Orange county
- 6111 = Ventura county

In [28]:
acquire_df.head()

,roomcnt,bedroomcnt,bathroomcnt,garagecarcnt,garagetotalsqft,yearbuilt,taxamount,fips,calculatedfinishedsquarefeet,taxvaluedollarcnt,lotsizesquarefeet,landtaxvaluedollarcnt,poolcnt
567,261.0,5,4.0,4.0,781.0,1977,16628.84,6111,4070,1464000,140698.0,1082000.0,1.0
573,261.0,4,3.5,3.0,705.0,1988,10336.12,6111,2475,826982,4974.0,330792.0,1.0
574,261.0,4,3.0,3.0,529.0,1988,10245.32,6111,2698,825978,5538.0,412989.0,1.0
578,261.0,4,2.0,2.0,437.0,1970,6479.46,6111,1833,497696,7577.0,199078.0,1.0
579,261.0,3,2.0,2.0,462.0,1970,8788.00,6111,1594,698729,8163.0,420821.0,1.0


In [29]:
# acquire_df = acquire_df.replace(to_replace={'6037':'LA County', '6059':'Orange County', '6111':'Ventura County'}, value=none)

In [30]:
acquire_df.rename(columns = {'bedroomcnt':'bedrooms', 'bathroomcnt':'bathrooms', 'calculatedfinishedsquarefeet':'sq_ft', 'taxvaluedollarcnt':'property_value', 'yearbuilt':'year_built', 'fips':'location'})

,roomcnt,bedrooms,bathrooms,garagecarcnt,garagetotalsqft,year_built,taxamount,location,sq_ft,property_value,lotsizesquarefeet,landtaxvaluedollarcnt,poolcnt
567,261.0,5,4.0,4.0,781.0,1977,16628.84,6111,4070,1464000,140698.0,1082000.0,1.0
573,261.0,4,3.5,3.0,705.0,1988,10336.12,6111,2475,826982,4974.0,330792.0,1.0
574,261.0,4,3.0,3.0,529.0,1988,10245.32,6111,2698,825978,5538.0,412989.0,1.0
578,261.0,4,2.0,2.0,437.0,1970,6479.46,6111,1833,497696,7577.0,199078.0,1.0
579,261.0,3,2.0,2.0,462.0,1970,8788.00,6111,1594,698729,8163.0,420821.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55217,261.0,5,4.5,4.0,625.0,2001,13192.10,6059,5070,1248282,11579.0,341734.0,1.0
55221,261.0,5,4.5,4.0,665.0,2000,10053.54,6059,3793,901884,9750.0,348751.0,1.0
55234,261.0,5,5.5,5.0,761.0,2001,41563.50,6059,7812,3893609,37113.0,2209964.0,1.0
55238,261.0,5,4.5,4.0,646.0,2000,10767.92,6059,3687,923000,7032.0,491975.0,1.0


In [31]:
(acquire_df['taxvaluedollarcnt'] / acquire_df['landtaxvaluedollarcnt']).mean()

2.164899513220206

In [32]:
acquire_df.isnull().sum()

roomcnt                         0
bedroomcnt                      0
bathroomcnt                     0
garagecarcnt                    0
garagetotalsqft                 0
yearbuilt                       0
taxamount                       0
fips                            0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
lotsizesquarefeet               0
landtaxvaluedollarcnt           0
poolcnt                         0
dtype: int64

In [33]:
acquire_df.garagecarcnt.value_counts()

2.0     1885
3.0     1171
4.0      357
1.0      128
5.0      119
6.0       33
7.0       14
8.0        6
9.0        4
19.0       1
Name: garagecarcnt, dtype: int64

##### Below is a "working document" version of my wrangle.py file, additions/changes TBD

In [34]:
import pandas as pd
import numpy as np
import os
from env import user, password, host

def get_db_url(database):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

"""
USAGE: 
Use `from wrangle import wrangle_zillow` at the top of your notebook.
This 
"""
def get_zillow_data():
    """Seeks to read the cached zillow.csv first """
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        return get_new_zillow_data()

def get_new_zillow_data():
    """Returns a dataframe of all 2017 properties that are Single Family Residential"""

    sql = """
    SELECT
    bedroomcnt,
    bathroomcnt,
    fullbathcnt
    garagecarcnt,
    yearbuilt,
    taxamount,
    fips,
    calculatedfinishedsquarefeet,
    taxvaluedollarcnt,
    lotsizesquarefeet,
    landtaxvaluedollarcnt
    FROM properties_2017
    JOIN propertylandusetype USING (propertylandusetypeid)
    JOIN predictions_2017 ON properties_2017.id = predictions_2017.id
    WHERE propertylandusedesc = "Single Family Residential"
    AND predictions_2017.transactiondate LIKE "2017%%"
    """
    df = pd.read_sql(sql, get_db_url('zillow'))
    return df

def handle_nulls(df):    
    # We keep 99.41% of the data after dropping nulls
    # round(df.dropna().shape[0] / df.shape[0], 4) returned .9941
    df = df.dropna()
    return df


def optimize_types(df):
    # Convert some columns to integers
    # fips, yearbuilt, and bedrooms can be integers
    df["fips"] = df["fips"].astype(int)
    df["yearbuilt"] = df["yearbuilt"].astype(int)
    df["bedroomcnt"] = df["bedroomcnt"].astype(int)    
    df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
    df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
    return df


def handle_outliers(df):
    """Manually handle outliers that do not represent properties likely for 99% of buyers and zillow visitors"""
    df = df[df.bathroomcnt <= 6]
    
    df = df[df.bedroomcnt <= 6]

    df = df[df.taxvaluedollarcnt < 2_000_000]

    return df

def clean_variables(df):
    # Drop 'taxamount' column (variable is inconsistent based on time and location of collected value, could lead to poor analysis)
    # Rename columns and 'fips' values to reflect actual location (to solidify column as categorical variable)
    df = df.drop(columns = 'taxamount')
    df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                              'bathroomcnt':'bathrooms', 
                              'calculatedfinishedsquarefeet':'sq_ft', 
                              'taxvaluedollarcnt':'home_value', 
                              'yearbuilt':'year_built', 
                              'fips':'location',
                              'fullbathcnt':'full_bathrooms',
                              'garagecarcnt':'garage_spaces',
                              'lotsizesquarefeet':'lot_sq_ft',
                              'landtaxvaluedollarcnt':'property_value'
                             })
    df.location = df.location.replace(to_replace={6037:'LA County', 6059:'Orange County', 6111:'Ventura County'})

    return df 

def wrangle_zillow():
    """
    Acquires Zillow data
    Handles nulls
    optimizes or fixes data types
    handles outliers w/ manual logic
    returns a clean dataframe
    """
    df = get_zillow_data()

    df = handle_nulls(df)

    df = optimize_types(df)

    df = handle_outliers(df)

    df = clean_variables(df)

    # df.to_csv("zillow.csv", index=False)

    return df

In [35]:
df_py_test = wrangle_zillow()

In [36]:
df_py_test.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54167 entries, 0 to 56078
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bedrooms        54167 non-null  int64  
 1   bathrooms       54167 non-null  float64
 2   garage_spaces   54167 non-null  float64
 3   year_built      54167 non-null  int64  
 4   location        54167 non-null  object 
 5   sq_ft           54167 non-null  int64  
 6   home_value      54167 non-null  int64  
 7   lot_sq_ft       54167 non-null  float64
 8   property_value  54167 non-null  float64
dtypes: float64(4), int64(4), object(1)
memory usage: 4.1+ MB


In [37]:
df_py_test.head()

,bedrooms,bathrooms,garage_spaces,year_built,location,sq_ft,home_value,lot_sq_ft,property_value
0,3,4.0,4.0,1934,LA County,2822,1538506,7093.0,1076955.0
1,3,3.0,3.0,1990,LA County,2815,1106839,7601.0,651740.0
2,4,2.0,2.0,1936,LA County,2386,196622,7609.0,72463.0
3,2,1.0,1.0,1948,LA County,2406,628434,6244.0,456614.0
4,2,1.0,1.0,1936,LA County,962,545000,3752.0,387200.0


In [38]:
df_py_test.shape

(54167, 9)

# Exploration / Exploratory Analysis / Statistical Testing

# Modeling

# Modeling Evaluation